# Supplementary Data: Agreement metrics on metaphor coding

## Metaphors considered harmful? An exploratory study of the effectiveness of functional metaphors for end-to-end encryption

### Albese Demjaha, Jonathan Spring, Ingolf Becker, Simon Parkin and M. Angela Sasse

{albese.demjaha.16, jonathan.spring.15, i.becker, s.parkin, a.sasse}@ucl.ac.uk

The full will appear in USEC 2018, 18-21 February 2018, San Diego, CA, USA

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
import itertools, collections

In [2]:
%matplotlib inline

In [3]:
e2equot = pd.read_csv("data/E2E-coding-toPublish.csv")

In [4]:
## test data from wikipedia: https://en.wikipedia.org/wiki/Krippendorff%27s_alpha
testData = pd.DataFrame({"A" : [np.nan, np.nan, np.nan, np.nan, np.nan, 3, 4, 1, 2, 1, 1, 3, 3, np.nan, 3],
                         "B" : [1, np.nan, 2, 1, 3, 3, 4, 3, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan],
                         "C" : [np.nan, np.nan, 2, 1, 3, 4, 4, np.nan, 2, 1, 1, 3, 3, np.nan, 4]})

In [5]:
testData = testData[(~testData.isnull()).sum(axis=1) > 1]
testData

,A,B,C
2,NaN,2.0,2.0
3,NaN,1.0,1.0
4,NaN,3.0,3.0
5,3.0,3.0,4.0
6,4.0,4.0,4.0
7,1.0,3.0,NaN
8,2.0,NaN,2.0
9,1.0,NaN,1.0
10,1.0,NaN,1.0
11,3.0,NaN,3.0


In [6]:
def calculateCoincidence(lboolMatrix, rboolMatrix):
    cbined = lboolMatrix & rboolMatrix
    divisors = (~cbined.isnull()).sum(axis=1) - 1
    scores = np.zeros(len(lboolMatrix))
    lboolMatrix = lboolMatrix.fillna(False)
    rboolMatrix = rboolMatrix.fillna(False)
    for c in lboolMatrix.columns:
        scores += lboolMatrix[c] * rboolMatrix[[x for x in rboolMatrix.columns if x != c]].sum(axis=1)
    scores = scores / divisors
    return scores.sum()
    

In [7]:
def createCoincidenceMatrix(units, categories, comparisonFun):
    coincidence = pd.DataFrame(data=np.zeros((len(categories), len(categories))), index=categories, columns=categories)
    for i in range(len(categories)):
        iboolsl = units.applymap(lambda x: x if pd.isnull(x) else comparisonFun(categories[i], x))
        coincidence.loc[categories[i], categories[i]] = calculateCoincidence(iboolsl, iboolsl)
        for j in range(i + 1, len(categories)):
            iboolsr = units.applymap(lambda x: x if pd.isnull(x) else comparisonFun(categories[j], x))
            score = calculateCoincidence(iboolsl, iboolsr)
            coincidence.loc[categories[i], categories[j]] = score
            coincidence.loc[categories[j], categories[i]] = score
    return coincidence

In [8]:
def getExpected(coinc):
    asum = coinc.sum()
    expected = pd.DataFrame((np.outer(asum, asum) - np.eye(len(asum))*asum.values) / (asum.sum() -1),index=coinc.index, 
                            columns=coinc.columns)
    return expected

In [9]:
def getAlphaNominal(coinc, expec):
    dob = (~np.eye(len(coinc),dtype=np.bool) * coinc).sum().sum()
    dex = (~np.eye(len(expec),dtype=np.bool) * expec).sum().sum()
    return 1 - dob/dex

In [10]:
def getAlpha(coinc, expec, scoringFun, printScoring=True):
    scoring = pd.DataFrame(dict((x, [scoringFun(x, y) for y in coinc.index]) for x in coinc.columns), index=coinc.index,
                          columns=coinc.columns)
    if printScoring:
        print(scoring)
    dob = (scoring*coinc).sum().sum()
    dex = (scoring*expec).sum().sum()
    return 1- dob/dex

In [11]:
testCoincidence = createCoincidenceMatrix(testData, [1, 2, 3, 4], lambda x,y: x == y)
testCoincidence

,1,2,3,4
1,6.0,0.0,1.0,0.0
2,0.0,4.0,0.0,0.0
3,1.0,0.0,7.0,2.0
4,0.0,0.0,2.0,3.0


In [12]:
getAlphaNominal(testCoincidence, getExpected(testCoincidence))

0.69135802469135799

In [13]:
getAlpha(testCoincidence, getExpected(testCoincidence), lambda x, y: (x - y)**2)

   1  2  3  4
1  0  1  4  9
2  1  0  1  4
3  4  1  0  1
4  9  4  1  0


0.81084489281210592

In [14]:
def scoreMetaphors(x, y):
    if x == y:
        return 0
    if x > y:
        a = x
        x = y
        y = a
    if x == "A":
        if y in ["B", "D"]: return 1
        if y in ["C", "E"]: return 2
        if y in ["F", "G"]: return 3
    if x == "B":
        if y in ["C", "D"]: return 1
        if y in ["E", "F"]: return 2
        if y in ["G"] : return 3
    if x == "C":
        if y in ["D", "E", "F"]: return 1
        if y in ["G"]: return 2
    if x == "D":
        if y in ["E"]: return 1
        if y in ["F"]: return 2
        if y in ["G"]: return 3
    if x == "E":
        if y in ["F"]: return 1
        if y in ["G"]: return 2
    if x == "F" and y == "G": return 1
            

In [15]:
codeColumns = ["Ingolf", "Jono", "Albesa"]

In [16]:
e2ecoinc = createCoincidenceMatrix(e2equot[codeColumns], list("ABCDEFG"), lambda x, y: x.lower() in y.lower())

In [17]:
e2ecoinc

,A,B,C,D,E,F,G
A,25.0,0.0,0.0,2.0,2.0,0.0,0.0
B,0.0,6.0,1.0,2.0,0.0,0.0,0.0
C,0.0,1.0,7.0,0.0,4.0,1.0,0.0
D,2.0,2.0,0.0,15.0,6.0,0.0,0.0
E,2.0,0.0,4.0,6.0,30.0,2.0,0.0
F,0.0,0.0,1.0,0.0,2.0,6.0,1.0
G,0.0,0.0,0.0,0.0,0.0,1.0,4.0


In [18]:
getAlphaNominal(e2ecoinc, getExpected(e2ecoinc))

0.61046511627906985

In [19]:
getAlpha(e2ecoinc, getExpected(e2ecoinc), scoreMetaphors)

   A  B  C  D  E  F  G
A  0  1  2  1  2  3  3
B  1  0  1  1  2  2  3
C  2  1  0  1  1  1  2
D  1  1  1  0  1  2  3
E  2  2  1  1  0  1  2
F  3  2  1  2  1  0  1
G  3  3  2  3  2  1  0


0.72254231184731721

In [20]:
encquot = pd.read_csv("data/Encryption-coding-toPublish.csv")

In [21]:
encquot.shape

(140, 4)

In [22]:
enccoinc = createCoincidenceMatrix(encquot[codeColumns], list("ABCDEFG"), lambda x, y: x.lower() in y.lower())
enccoinc

,A,B,C,D,E,F,G
A,67.0,0.0,2.0,7.0,2.0,0.0,0.0
B,0.0,12.0,8.0,6.0,4.0,2.0,0.0
C,2.0,8.0,19.0,5.0,5.0,0.0,0.0
D,7.0,6.0,5.0,77.0,22.0,3.0,0.0
E,2.0,4.0,5.0,22.0,79.0,6.0,0.0
F,0.0,2.0,0.0,3.0,6.0,14.0,1.0
G,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [23]:
getAlphaNominal(enccoinc, getExpected(enccoinc))

0.54922775593200068

In [24]:
getAlpha(enccoinc, getExpected(enccoinc), scoreMetaphors)

   A  B  C  D  E  F  G
A  0  1  2  1  2  3  3
B  1  0  1  1  2  2  3
C  2  1  0  1  1  1  2
D  1  1  1  0  1  2  3
E  2  2  1  1  0  1  2
F  3  2  1  2  1  0  1
G  3  3  2  3  2  1  0


0.61195236534469577

In [25]:
combined = pd.concat([e2equot, encquot])

In [26]:
combined = combined.reset_index(drop=True)

In [27]:
comcoinc = createCoincidenceMatrix(combined[codeColumns], list("ABCDEFG"), lambda x, y: x.lower() in y.lower())

In [28]:
getAlphaNominal(comcoinc, getExpected(comcoinc))

0.56523720702548041

In [29]:
getAlpha(comcoinc, getExpected(comcoinc), scoreMetaphors, printScoring=False)

0.6428003324901479

In [30]:
#pairwise:
for i in range(len(codeColumns)):
    for j in range(i+1, len(codeColumns)):
        selcoinc = createCoincidenceMatrix(combined[[codeColumns[i], codeColumns[j]]], 
                                           list("ABCDEFG"), lambda x, y: x.lower() in y.lower())
        expected = getExpected(selcoinc)
        print("%6s vs %6s: nominal alpha: %.4f, ordinal alpha %.4f" %(codeColumns[i], codeColumns[j],
                                                                   getAlphaNominal(selcoinc, expected),
                                                                   getAlpha(selcoinc, expected, scoreMetaphors, 
                                                                            printScoring=False)))

Ingolf vs   Jono: nominal alpha: 0.3618, ordinal alpha 0.4788
Ingolf vs Albesa: nominal alpha: 0.5220, ordinal alpha 0.6216
  Jono vs Albesa: nominal alpha: 0.8210, ordinal alpha 0.8398


In [31]:
codes = list("ABCDEFG")

In [32]:
codeScores = {}
for i in range(len(codes)):
    codeScores[codes[i]] = combined[codeColumns].applymap(lambda x: (not pd.isnull(x)) and codes[i].lower() 
                                                          in x.lower()).sum(1)
codeScores = pd.DataFrame(codeScores)

In [33]:
rowMax = codeScores.max(1)

In [34]:
(codeScores.eq(rowMax, axis='rows').sum(1) * (rowMax > 1)).value_counts()

1    183
0      2
dtype: int64

In [35]:
## great, no ties!

In [36]:
maxScore = codeScores.eq(rowMax, axis='rows').apply(lambda x: x.index[x].values[0], axis=1)

In [37]:
maxScore = maxScore[(rowMax > 1)]

In [38]:
len(maxScore)

183

In [39]:
maxScore.value_counts()

E    59
D    52
A    34
C    14
B    12
F    10
G     2
dtype: int64

In [40]:
e2evcs = maxScore[:45].value_counts()

In [41]:
encvcs = maxScore[45:].value_counts()

In [42]:
vcs = pd.DataFrame({"e2e" : e2evcs, "enc" : encvcs}).fillna(0).astype(np.int64)

In [43]:
print(vcs.T)

      A   B  C   D   E  F  G
e2e   9   2  5   9  16  2  2
enc  25  10  9  43  43  8  0
